In [1]:
import numpy as np
import pandas as pd
import keras
from sklearn import preprocessing
from sklearn import metrics
from keras import backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, SimpleRNN, RNN, LSTM, GRU, SpatialDropout1D, Dropout

In [2]:
import matplotlib.pyplot as plt
import os, os.path, sys
import copy

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
path = r"/content/drive/MyDrive/Colab Notebooks/0k/X"

In [5]:
dir = os.listdir(path)

In [6]:
file_names = []
for file in dir:
    file_names.append(file)

In [7]:
test_file_names = []
train_file_names = copy.copy(file_names)
file_indx = np.sort(np.random.randint(1,30,5))[::-1]

for ind in file_indx:
    test_file_names.append(file_names[ind])
    train_file_names.pop(ind)

In [120]:
min_kord = [float('inf') for x in range(61000)]
max_kord = [float('-inf') for x in range(61000)]

for file in file_names:
  current_file = pd.read_csv(path+'/'+file, header=None)[:61000].astype('int')

  for i in range(len(current_file)):
    if current_file[0][i] < min_kord[i]:
      min_kord[i] = current_file[0][i]
    elif current_file[0][i] > max_kord[i]:
      max_kord[i] = current_file[0][i]

Potencijalni time_stepovi su (1, 2, 4, 5, 8, 10, 20, 25, 40, 50, 61, 100), kako bi bili deljivi sa 61000

In [10]:
train_ = np.concatenate([pd.read_csv(path + "/" + file, header=None)[:61000].astype('int')
                              for file in train_file_names], axis=0)
test_ = np.concatenate([pd.read_csv(path + "/" + file, header=None)[:61000].astype('int')
                              for file in test_file_names], axis=0)

In [166]:
scaler = preprocessing.MinMaxScaler(feature_range=(0,100))

In [33]:
#scaler = preprocessing.StandardScaler()

In [167]:
scaler.fit(train_)

MinMaxScaler(feature_range=(0, 100))

In [168]:
train_ = scaler.transform(train_)
test_ = scaler.transform(test_)

In [169]:
min_kord_scaled = scaler.transform(np.array(min_kord).reshape(-1,1))
max_kord_scaled = scaler.transform(np.array(max_kord).reshape(-1,1))

best step = 20

In [170]:
def create_timeseries_split(data, time_steps):

  # Get the number of samples in the data
  num_samples = len(data) - time_steps

  # Create empty arrays to store features and target values
  x_train = np.zeros((num_samples, time_steps, 1))
  y_train = np.zeros((num_samples, 1))

  # Loop through the data and create features and target values
  for i in range(num_samples):
    # Extract a slice of data for the current feature
    x_train[i] = data[i:i+time_steps, :]

    # The target value is the next value after the feature
    y_train[i] = data[i+time_steps, 0]

  return x_train, y_train

In [173]:
time_steps = 10

In [174]:
train_X, train_y = create_timeseries_split(train_, time_steps)

In [179]:
test_X, test_y = create_timeseries_split(test_, time_steps)

In [180]:
np.random.seed(1234)

In [181]:
input_shape = (time_steps,1)

In [ ]:
units1 = [16,32]
units2 = [16,32]
units3 = [16,32]
units4 = [16,32]
units5 = [16,32]

In [ ]:
best_skor = float('inf')
best_unit1 = None
best_unit2 = None
best_unit3 = None



for unit1 in units1:
  for unit2 in units2:
    for unit3 in units3:
      for unit4 in units4:
        for unit5 in units5:
          model = Sequential()
          model.add(LSTM(units = unit1, input_shape = input_shape, activation = keras.layers.LeakyReLU()))
          model.add(Dense(units = unit2, activation=keras.layers.LeakyReLU()))
          model.add(Dense(units = unit3, activation=keras.layers.LeakyReLU()))
          model.add(Dense(units = unit4, activation=keras.layers.LeakyReLU()))
          model.add(Dense(units = unit5, activation=keras.layers.LeakyReLU()))
          model.add(Dense(units = 1, activation=keras.layers.LeakyReLU(), bias_initializer='zeros', kernel_initializer='normal'))
          model.compile(loss='mean_squared_error', optimizer = 'adam')

          model.fit(train_X, train_y, epochs = 10, batch_size = 128, verbose = 2)
          train_scores = model.evaluate(test_X, test_y)

          if train_scores < best_skor:
            best_skor = train_scores
            best_unit1 = unit1
            best_unit2 = unit2
            best_unit3 = unit3
            best_unit4 = unit4
            best_unit5 = unit5
print('best_skor: ', best_skor)
print('best_unit1: ', best_unit1)
print('best_unit2: ', best_unit2)
print('best_unit3: ', best_unit3)
print('best_unit4: ', best_unit4)
print('best_unit5: ', best_unit5)

Epoch 1/10
11915/11915 - 34s - loss: 56760624.0000 - 34s/epoch - 3ms/step
Epoch 2/10
11915/11915 - 32s - loss: 1053319.7500 - 32s/epoch - 3ms/step
Epoch 3/10
11915/11915 - 31s - loss: 529635.6875 - 31s/epoch - 3ms/step
Epoch 4/10
11915/11915 - 31s - loss: 338038.4062 - 31s/epoch - 3ms/step
Epoch 5/10
11915/11915 - 30s - loss: 343894.4375 - 30s/epoch - 3ms/step
Epoch 6/10
11915/11915 - 30s - loss: 625263.8750 - 30s/epoch - 3ms/step
Epoch 7/10
11915/11915 - 32s - loss: 272882.4062 - 32s/epoch - 3ms/step
Epoch 8/10
11915/11915 - 30s - loss: 4515911.5000 - 30s/epoch - 3ms/step
Epoch 9/10
11915/11915 - 30s - loss: 1354136.2500 - 30s/epoch - 3ms/step
Epoch 10/10
11915/11915 - 31s - loss: 18459.5352 - 31s/epoch - 3ms/step
9532/9532 [==============================] - 18s 2ms/step - loss: 4123.0361
Epoch 1/10
11915/11915 - 36s - loss: 34927076.0000 - 36s/epoch - 3ms/step
Epoch 2/10
11915/11915 - 31s - loss: 1132236.8750 - 31s/epoch - 3ms/step
Epoch 3/10
11915/11915 - 31s - loss: 34188728.0000 -

NameError: name 'best_unit5' is not defined

In [ ]:
best_skor:  238.99697875976562
best_unit1:  32
best_unit2:  16
best_unit3:  32
best_unit4:  32
best_unit5:  16

Epoch 19/40
47657/47657 - 122s - loss: 500956.6250 - 122s/epoch - 3ms/step
Epoch 20/40
47657/47657 - 125s - loss: 69292.7344 - 125s/epoch - 3ms/step
Epoch 21/40
47657/47657 - 125s - loss: 1439553.7500 - 125s/epoch - 3ms/step

In [182]:
optimizer = keras.optimizers.Adam(learning_rate=0.00001)

In [183]:
model = Sequential()
model.add(LSTM(units = 32, input_shape = input_shape, activation = keras.layers.LeakyReLU()))
model.add(Dense(units = 16, activation=keras.layers.LeakyReLU()))
model.add(Dense(units = 32, activation=keras.layers.LeakyReLU()))
model.add(Dense(units = 32, activation=keras.layers.LeakyReLU()))
#model.add(Dense(units = unit5, activation=keras.layers.LeakyReLU()))
model.add(Dense(units = 1, activation=keras.layers.LeakyReLU(), bias_initializer='zeros', kernel_initializer='normal'))
model.compile(loss='mean_squared_error', optimizer = 'adam')


In [184]:
history = model.fit(train_X, train_y, epochs=100, batch_size = 64, verbose = 2)

Epoch 1/100
23828/23828 - 156s - loss: 5.9193 - 156s/epoch - 7ms/step
Epoch 2/100
23828/23828 - 155s - loss: 0.0116 - 155s/epoch - 7ms/step
Epoch 3/100
23828/23828 - 149s - loss: 0.0094 - 149s/epoch - 6ms/step
Epoch 4/100
23828/23828 - 150s - loss: 0.0284 - 150s/epoch - 6ms/step
Epoch 5/100
23828/23828 - 150s - loss: 0.0064 - 150s/epoch - 6ms/step
Epoch 6/100
23828/23828 - 151s - loss: 0.0067 - 151s/epoch - 6ms/step
Epoch 7/100
23828/23828 - 155s - loss: 0.0058 - 155s/epoch - 6ms/step
Epoch 8/100
23828/23828 - 158s - loss: 0.0053 - 158s/epoch - 7ms/step
Epoch 9/100
23828/23828 - 156s - loss: 0.0050 - 156s/epoch - 7ms/step
Epoch 10/100
23828/23828 - 150s - loss: 0.0042 - 150s/epoch - 6ms/step
Epoch 11/100
23828/23828 - 149s - loss: 0.0041 - 149s/epoch - 6ms/step
Epoch 12/100
23828/23828 - 149s - loss: 0.0040 - 149s/epoch - 6ms/step
Epoch 13/100
23828/23828 - 150s - loss: 0.0033 - 150s/epoch - 6ms/step
Epoch 14/100
23828/23828 - 151s - loss: 0.0032 - 151s/epoch - 6ms/step
Epoch 15/100
23

In [147]:
train_scores = model.evaluate(test_X, test_y)

9532/9532 [==============================] - 23s 2ms/step - loss: 2.6647e-07


In [148]:
train_predict = model.predict(train_X)
test_predict = model.predict(test_X)

9532/9532 [==============================] - 22s 2ms/step


In [149]:
n = 61000
test_size = len(test_file_names)
avg_koord = []
for i in range(n):
    avg_koord.append(sum(test_[i] + test_[n+i] + test_[2*n + i] + test_[3*n + i] + test_[4*n + i])/test_size)


In [50]:
#test_predict = scaler.inverse_transform(test_predict)
#train_predict = scaler.inverse_transform(train_predict)

In [23]:
num_correct_train = 0
for i in range(n):
    if (avg_koord[i]-avg_koord[i]*0.0003) <= train_predict[i][0] <= (avg_koord[i]+avg_koord[i]*0.0003):
        num_correct_train += 1
num_correct_train

2641

In [24]:
num_correct_test = 0
for i in range(n):
    if (avg_koord[i]-avg_koord[i]*0.0003) <= test_predict[i][0] <= (avg_koord[i]+avg_koord[i]*0.0003):
        num_correct_test += 1
num_correct_test

2987

In [151]:
num_correct_pojas_test = 0
for i in range(n):
  if(min_kord_scaled[i] <= test_predict[i][0] <= max_kord_scaled[i]):
    num_correct_pojas_test += 1
(num_correct_pojas_test/n)

0.16234426229508198

In [152]:
test_predict_unscaled = scaler.inverse_transform(test_predict)

In [153]:
for i in range(n):
  if(min_kord_scaled[i] <= test_predict[i][0] <= max_kord_scaled[i]):
    print(min_kord[i], test_predict_unscaled[i][0],max_kord[i], 'index: ', i)


9955 9972.618 9973 index:  4310
9945 9965.196 9971 index:  4311
9935 9955.97 9968 index:  4312
9928 9948.571 9960 index:  4313
9925 9942.946 9949 index:  4314
9906 9929.946 9939 index:  4315
9892 9913.845 9936 index:  4316
9888 9900.384 9928 index:  4317
9881 9892.642 9919 index:  4318
9867 9871.493 9904 index:  4319
9793 9799.571 9828 index:  4324
9774 9792.571 9805 index:  4325
9751 9773.188 9787 index:  4326
9743 9746.603 9775 index:  4327
9601 9606.704 9635 index:  4334
9578 9584.837 9606 index:  4335
112 112.837234 146 index:  4572
99 112.98567 129 index:  4573
75 103.59505 126 index:  4574
63 84.72005 115 index:  4575
48 57.399734 75 index:  4579
0 32.141922 38 index:  4583
3 18.10286 38 index:  4584
6 14.970046 32 index:  4585
0 18.266922 20 index:  4586
179942 179989.88 179999 index:  8432
179960 180003.61 180005 index:  8433
179973 180017.19 180024 index:  8439
179989 180014.88 180031 index:  8440
179999 180018.89 180039 index:  8441
180001 180023.38 180040 index:  8442
179996

KeyboardInterrupt: 

In [164]:
for i in range(n):
  if(min_kord_scaled[i] <= test_predict[i][0] <= max_kord_scaled[i]):
    print(min_kord_scaled[i], test_predict[i][0],max_kord_scaled[i], 'index: ', i)


[-1.06448333] -1.06422 [-1.06421423] index:  4310
[-1.06463283] -1.0643309 [-1.06424413] index:  4311
[-1.06478233] -1.0644689 [-1.06428898] index:  4312
[-1.06488698] -1.0645795 [-1.06440858] index:  4313
[-1.06493183] -1.0646635 [-1.06457303] index:  4314
[-1.06521589] -1.0648578 [-1.06472253] index:  4315
[-1.06542519] -1.0650986 [-1.06476738] index:  4316
[-1.06548499] -1.0652999 [-1.06488698] index:  4317
[-1.06558964] -1.0654156 [-1.06502154] index:  4318
[-1.06579894] -1.0657318 [-1.06524579] index:  4319
[-1.06690525] -1.066807 [-1.06638199] index:  4324
[-1.0671893] -1.0669117 [-1.06672585] index:  4325
[-1.06753315] -1.0672014 [-1.06699495] index:  4326
[-1.06765275] -1.0675989 [-1.06717435] index:  4327
[-1.06977566] -1.0696903 [-1.06926736] index:  4334
[-1.07011952] -1.0700173 [-1.06970091] index:  4335
[-1.21163704] -1.2116245 [-1.21112873] index:  4572
[-1.21183139] -1.2116224 [-1.21138288] index:  4573
[-1.21219019] -1.2117627 [-1.21142773] index:  4574
[-1.21236959] -1

KeyboardInterrupt: 

In [165]:
for i in range(n):
  print(min_kord_scaled[i], test_predict[i][0],max_kord_scaled[i], 'index: ', i)


[-1.2133264] -1.212568 [-1.21331145] index:  0
[-1.2133264] -1.2124455 [-1.21331145] index:  1
[-1.2133264] -1.2123258 [-1.2132965] index:  2
[-1.21331145] -1.2121747 [-1.21323669] index:  3
[-1.21328155] -1.2120792 [-1.21313204] index:  4
[-1.21320679] -1.2121543 [-1.21299749] index:  5
[-1.21310214] -1.2122171 [-1.21284799] index:  6
[-1.21301244] -1.2121317 [-1.21274334] index:  7
[-1.21293769] -1.2120308 [-1.21275829] index:  8
[-1.21292274] -1.2118517 [-1.21272839] index:  9
[-1.21287789] -1.211601 [-1.21272839] index:  10
[-1.21280314] -1.2113912 [-1.21266859] index:  11
[-1.21271344] -1.2112315 [-1.21250414] index:  12
[-1.21263869] -1.2111092 [-1.21230979] index:  13
[-1.21250414] -1.211087 [-1.21220514] index:  14
[-1.21241444] -1.2109891 [-1.21189119] index:  15
[-1.21227989] -1.2107934 [-1.21169684] index:  16
[-1.21208554] -1.2105291 [-1.21162209] index:  17
[-1.21195099] -1.2103032 [-1.21157724] index:  18
[-1.21175664] -1.210008 [-1.21153238] index:  19
[-1.21159219] -1.2

KeyboardInterrupt: 

In [133]:
for i in range(n):
    print((min_kord[i], test_predict_unscaled[i][0], max_kord[i]), 'index:',i)

(-1, -148.77214, 0) index: 0
(-1, -148.77214, 0) index: 1
(-1, -147.60808, 1) index: 2
(0, -143.02214, 5) index: 3
(2, -136.59245, 12) index: 4
(7, -129.29558, 21) index: 5
(14, -119.79558, 31) index: 6
(20, -113.006516, 38) index: 7
(25, -114.162766, 37) index: 8
(26, -115.35808, 39) index: 9
(29, -111.6237, 39) index: 10
(34, -107.38933, 43) index: 11
(40, -97.381516, 54) index: 12
(45, -83.4987, 67) index: 13
(54, -70.2487, 74) index: 14
(60, -58.201828, 95) index: 15
(69, -48.342453, 108) index: 16
(82, -44.08464, 113) index: 17
(91, -36.623703, 116) index: 18
(104, -25.95964, 119) index: 19
(115, -11.053391, 135) index: 20
(124, 3.2044215, 146) index: 21
(133, 22.188797, 158) index: 22
(150, 49.97786, 181) index: 23
(181, 78.82161, 207) index: 24
(209, 89.555984, 234) index: 25
(227, 98.07161, 244) index: 26
(237, 101.28255, 252) index: 27
(241, 110.11067, 256) index: 28
(246, 125.32942, 263) index: 29
(257, 150.24348, 280) index: 30
(277, 178.3763, 306) index: 31
(307, 209.39192,

KeyboardInterrupt: 